In [ ]:
#@title Mô phỏng Bảo mật Tin nhắn Âm thanh (Đã được chú thích chi tiết)

# ==============================================================================
# PHẦN 1: CÀI ĐẶT VÀ CẤU HÌNH
# ==============================================================================
print("⏳ Bước 1: Cài đặt các thư viện cần thiết...")
# Cài đặt các thư viện Python cần thiết một cách "âm thầm" (-q: quiet).
# - flask: Một micro-framework để tạo máy chủ web bằng Python.
# - pyngrok: Thư viện để tạo một đường hầm (tunnel) an toàn từ máy chủ cục bộ (localhost) ra Internet công cộng bằng dịch vụ ngrok.
# - cryptography: Thư viện cốt lõi để thực hiện các thao tác mã hóa phức tạp như RSA và AES.
!pip install flask pyngrok cryptography -q
print("✅ Cài đặt hoàn tất.")

print("🔑 Bước 2: Cấu hình token ngrok...")
# Cấu hình token xác thực cho ngrok.
# ngrok yêu cầu một authtoken để liên kết phiên chạy với tài khoản của bạn,
# cho phép các kết nối ổn định và các tính năng nâng cao hơn.
!ngrok authtoken 2vQiHuSkEACuMQdfLaOTROncorU_3gmW5HFXANyHBHUfm2ufF
print("✅ Token đã được cấu hình.")

# ==============================================================================
# PHẦN 2: MÃ NGUỒN ỨNG DỤNG (BACKEND + FRONTEND)
# ==============================================================================
# --- Các thư viện cần thiết cho Backend ---
import os
import base64
import json
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
from cryptography.hazmat.primitives.asymmetric import rsa, padding as rsa_padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

# Khởi tạo ứng dụng Flask, đây là trái tim của backend.
app = Flask(__name__)

# --- Giao diện (Frontend) với trình phát Audio ---
# Toàn bộ giao diện người dùng được chứa trong một chuỗi đa dòng của Python.
# Flask sẽ gửi chuỗi HTML này đến trình duyệt của người dùng.
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Mô phỏng Bảo mật Tin nhắn Âm thanh</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif; line-height: 1.6; padding: 20px; background-color: #f4f7f9; color: #333; }
        .container { max-width: 1200px; margin: auto; background: white; padding: 25px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); }
        h1, h2 { color: #0056b3; border-bottom: 2px solid #eef; padding-bottom: 10px; }
        .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 30px; }
        .box { padding: 20px; border: 1px solid #ddd; border-radius: 5px; background-color: #fafafa; }
        textarea { width: 100%; height: 150px; padding: 10px; border: 1px solid #ccc; border-radius: 4px; font-family: monospace; font-size: 13px; box-sizing: border-box; }
        button { background-color: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 6px; cursor: pointer; font-size: 16px; margin-top: 10px; transition: background-color 0.3s; }
        button:hover { background-color: #0056b3; }
        button:disabled { background-color: #ccc; cursor: not-allowed; }
        #stop-recording-btn { background-color: #dc3545; }
        #stop-recording-btn:hover { background-color: #c82333; }
        pre { background-color: #eef; padding: 15px; border-radius: 4px; white-space: pre-wrap; word-wrap: break-word; font-size: 13px; max-height: 300px; overflow-y: auto; }
        label { font-weight: bold; margin-bottom: 5px; display: block; }
        .full-width { grid-column: 1 / -1; }
        #recording-status { margin-top: 15px; font-weight: bold; color: #17a2b8; }
        /* Style cho trình phát audio, ban đầu sẽ được ẩn đi. */
        #audio-playback-container { margin-top: 20px; display: none; }
        audio { width: 100%; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Mô phỏng Truyền tin Âm thanh An toàn (AES-256 & RSA-2048)</h1>
        <div class="box full-width">
            <h2>Bước 1: Tạo khóa</h2>
            <p>Nhấn nút để tạo các cặp khóa RSA cho Alice (Người gửi) và Bob (Người nhận).</p>
            <button id="generate-keys-btn">Tạo Khóa cho cả Alice và Bob</button>
        </div>
        <div class="grid">
            <div class="box">
                <h2>Phía Người gửi (Alice)</h2>
                <label for="alice-public-key">Khóa công khai của Alice:</label>
                <textarea id="alice-public-key" readonly></textarea>
                <label for="alice-private-key">Khóa riêng của Alice:</label>
                <textarea id="alice-private-key" readonly></textarea>
                <hr>
                <label for="bob-public-key-input">Khóa công khai của Bob (đã tự điền):</label>
                <textarea id="bob-public-key-input"></textarea>
                <hr>
                <label>Ghi âm Tin nhắn thoại:</label>
                <div>
                    <button id="start-recording-btn" disabled>Bắt đầu Ghi âm</button>
                    <button id="stop-recording-btn" disabled>Dừng Ghi âm</button>
                </div>
                <div id="recording-status">Trạng thái: Sẵn sàng. (Vui lòng tạo khóa trước)</div>
                <div id="audio-playback-container">
                    <label>Nghe lại bản ghi âm đã gửi:</label>
                    <audio id="audio-playback" controls></audio>
                </div>
            </div>
            <div class="box">
                <h2>Phía Người nhận (Bob)</h2>
                <label for="bob-public-key">Khóa công khai của Bob:</label>
                <textarea id="bob-public-key" readonly></textarea>
                <label for="bob-private-key">Khóa riêng của Bob:</label>
                <textarea id="bob-private-key" readonly></textarea>
            </div>
        </div>
        <div class="box full-width">
            <h2>Kết quả Xử lý</h2>
            <h3>Gói tin được gửi đi (JSON):</h3>
            <pre id="packet-output">Gói tin sẽ xuất hiện ở đây...</pre>
            <h3>Nhật ký xử lý (Log):</h3>
            <pre id="log-output">Nhật ký các bước sẽ xuất hiện ở đây...</pre>
        </div>
    </div>

    <script>
        // --- Lấy tham chiếu đến các phần tử HTML để điều khiển chúng ---
        const generateBtn = document.getElementById('generate-keys-btn');
        const startRecordingBtn = document.getElementById('start-recording-btn');
        const stopRecordingBtn = document.getElementById('stop-recording-btn');
        const statusDiv = document.getElementById('recording-status');
        const audioPlayback = document.getElementById('audio-playback');
        const audioPlaybackContainer = document.getElementById('audio-playback-container');
        // ... và các ô textarea, khu vực hiển thị khác.

        // --- Biến cho việc ghi âm ---
        let mediaRecorder; // Đối tượng MediaRecorder để ghi âm.
        let audioChunks = []; // Mảng để lưu các đoạn âm thanh được ghi.

        // --- Xử lý sự kiện khi người dùng nhấn nút "Tạo Khóa" ---
        generateBtn.addEventListener('click', async () => {
            logOutput.textContent = 'Đang tạo khóa...';
            try {
                // Gọi đến API '/generate-all-keys' trên server bằng phương thức POST.
                const response = await fetch('/generate-all-keys', { method: 'POST' });
                const data = await response.json(); // Nhận dữ liệu JSON chứa các khóa.

                // Điền các khóa nhận được vào các ô textarea tương ứng.
                document.getElementById('alice-public-key').value = data.alice_public_key;
                document.getElementById('alice-private-key').value = data.alice_private_key;
                document.getElementById('bob-public-key').value = data.bob_public_key;
                document.getElementById('bob-private-key').value = data.bob_private_key;
                document.getElementById('bob-public-key-input').value = data.bob_public_key;

                logOutput.textContent = 'Đã tạo khóa thành công! Sẵn sàng để ghi âm.';
                startRecordingBtn.disabled = false; // Kích hoạt nút ghi âm.
            } catch (error) {
                logOutput.textContent = `Lỗi khi tạo khóa: ${error.message}.`;
            }
        });

        // --- Xử lý sự kiện khi nhấn nút "Bắt đầu Ghi âm" ---
        startRecordingBtn.addEventListener('click', async () => {
            try {
                // Sử dụng API của trình duyệt để yêu cầu quyền truy cập micro.
                const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                mediaRecorder = new MediaRecorder(stream); // Khởi tạo đối tượng ghi âm.
                audioChunks = []; // Reset mảng chứa dữ liệu âm thanh.
                audioPlaybackContainer.style.display = 'none'; // Ẩn trình phát cũ đi.

                // Sự kiện này được kích hoạt liên tục khi có dữ liệu âm thanh mới.
                mediaRecorder.ondataavailable = event => {
                    audioChunks.push(event.data);
                };

                // Sự kiện này được kích hoạt khi việc ghi âm dừng lại.
                mediaRecorder.onstop = async () => {
                    // Ghép các đoạn âm thanh lại thành một file hoàn chỉnh (dưới dạng Blob).
                    const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });

                    // Tạo một URL tạm thời cho file âm thanh vừa ghi và gán vào trình phát <audio>
                    // để người dùng có thể nghe lại ngay lập tức.
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioPlayback.src = audioUrl;
                    audioPlaybackContainer.style.display = 'block'; // Hiển thị trình phát.

                    // Tạo đối tượng FormData để gửi cả file âm thanh và các khóa (dạng text)
                    // đến máy chủ trong cùng một yêu cầu POST.
                    const formData = new FormData();
                    formData.append('audio_data', audioBlob, 'recording.webm');
                    formData.append('alice_private_key', document.getElementById('alice-private-key').value);
                    formData.append('alice_public_key', document.getElementById('alice-public-key').value);
                    formData.append('bob_public_key', document.getElementById('bob-public-key-input').value);
                    formData.append('bob_private_key', document.getElementById('bob-private-key').value); // Gửi cả khóa riêng của Bob để mô phỏng giải mã.

                    logOutput.textContent = 'Đã dừng ghi âm. Đang gửi dữ liệu để mã hóa và xử lý...';

                    // Gửi yêu cầu đến API '/process-message' với FormData làm body.
                    const response = await fetch('/process-message', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json(); // Nhận kết quả xử lý từ server.

                    // Hiển thị gói tin và nhật ký xử lý nhận được.
                    if (result.success) {
                        document.getElementById('packet-output').textContent = JSON.stringify(result.packet, null, 2);
                    } else {
                        document.getElementById('packet-output').textContent = 'Có lỗi xảy ra.';
                    }
                    logOutput.textContent = result.log;
                };

                // Bắt đầu quá trình ghi âm.
                mediaRecorder.start();
                statusDiv.textContent = "🔴 Đang ghi âm...";
                startRecordingBtn.disabled = true;
                stopRecordingBtn.disabled = false;

            } catch (err) {
                statusDiv.textContent = `Lỗi: ${err.message}. Bạn đã cho phép truy cập micro chưa?`;
            }
        });

        // --- Xử lý sự kiện khi nhấn nút "Dừng Ghi âm" ---
        stopRecordingBtn.addEventListener('click', () => {
            mediaRecorder.stop(); // Dừng MediaRecorder, việc này sẽ tự động kích hoạt sự kiện 'onstop' ở trên.
            statusDiv.textContent = "Trạng thái: Sẵn sàng ghi âm.";
            startRecordingBtn.disabled = false;
            stopRecordingBtn.disabled = true;
        });
    </script>
</body>
</html>
"""

# --- LOGIC MÁY CHỦ (BACKEND) ---

# Hàm này tạo ra một cặp khóa RSA-2048.
# - Private key: Khóa bí mật, phải được giữ kín. Dùng để giải mã và tạo chữ ký.
# - Public key: Khóa công khai, có thể chia sẻ. Dùng để mã hóa và xác thực chữ ký.
# Các khóa được trả về ở định dạng PEM, một định dạng văn bản tiêu chuẩn.
def generate_key_pair():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    pem_private = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ).decode('utf-8')
    pem_public = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ).decode('utf-8')
    return pem_private, pem_public

# Hàm này chuyển đổi các chuỗi khóa PEM (văn bản) trở lại thành các đối tượng khóa
# mà thư viện cryptography có thể sử dụng được.
def deserialize_keys(pem_private_key, pem_public_key):
    private_key = serialization.load_pem_private_key(pem_private_key.encode('utf-8'), password=None) if pem_private_key else None
    public_key = serialization.load_pem_public_key(pem_public_key.encode('utf-8')) if pem_public_key else None
    return private_key, public_key

# Route (đường dẫn) gốc của trang web.
# Khi người dùng truy cập vào URL chính, hàm này sẽ trả về giao diện HTML.
@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

# API endpoint để tạo khóa cho cả Alice và Bob.
# Khi nút "Tạo Khóa" được nhấn trên giao diện, một yêu cầu POST sẽ được gửi đến đây.
@app.route('/generate-all-keys', methods=['POST'])
def generate_all_keys_api():
    alice_private, alice_public = generate_key_pair()
    bob_private, bob_public = generate_key_pair()
    # Máy chủ sẽ tạo 2 cặp khóa và trả về dưới dạng JSON.
    return jsonify({
        "alice_public_key": alice_public, "alice_private_key": alice_private,
        "bob_public_key": bob_public, "bob_private_key": bob_private
    })

# API endpoint chính, xử lý toàn bộ quá trình mã hóa và giải mã mô phỏng.
# Khi người dùng dừng ghi âm, dữ liệu âm thanh và các khóa sẽ được gửi đến đây.
@app.route('/process-message', methods=['POST'])
def process_message_api():
    log = []  # Một danh sách để ghi lại từng bước xử lý.
    try:
        # Lấy file âm thanh và các chuỗi khóa từ request gửi lên từ frontend.
        audio_file = request.files.get('audio_data')
        if not audio_file:
            raise ValueError("Không nhận được dữ liệu âm thanh.")

        alice_pem_private = request.form.get('alice_private_key')
        alice_pem_public = request.form.get('alice_public_key')
        bob_pem_public = request.form.get('bob_public_key')
        bob_pem_private = request.form.get('bob_private_key')

        audio_message_bytes = audio_file.read()

        # =======================================================
        # Bắt đầu mô phỏng phía Người gửi (Alice)
        # =======================================================
        log.append("--- Bắt đầu phía Người gửi (Alice) ---")

        # Chuyển đổi chuỗi khóa thành đối tượng khóa.
        alice_private_key, _ = deserialize_keys(alice_pem_private, None)
        _, bob_public_key = deserialize_keys(None, bob_pem_public)

        # 1. Alice: Tạo khóa phiên AES-256 ngẫu nhiên. Đây là khóa đối xứng, rất nhanh để mã hóa dữ liệu lớn.
        aes_key = os.urandom(32)
        log.append("1. Alice: Tạo khóa phiên AES-256 ngẫu nhiên.")

        # 2. Alice: Mã hóa khóa AES bằng khóa công khai của Bob (Sử dụng RSA-OAEP).
        #    Điều này đảm bảo chỉ Bob (người có khóa riêng tương ứng) mới có thể giải mã được khóa AES này.
        encrypted_aes_key = bob_public_key.encrypt(
            aes_key,
            rsa_padding.OAEP(mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        log.append("2. Alice: Mã hóa khóa AES bằng khóa công khai của Bob.")

        # 3. Alice: Mã hóa tin nhắn thoại bằng khóa AES vừa tạo (Chế độ CBC).
        iv = os.urandom(16) # Tạo một vector khởi tạo (IV) ngẫu nhiên để tăng tính bảo mật.
        cipher_aes = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
        padder = PKCS7(algorithms.AES.block_size).padder() # Thêm đệm để dữ liệu đủ độ dài khối.
        padded_data = padder.update(audio_message_bytes) + padder.finalize()
        ciphertext = cipher_aes.encryptor().update(padded_data)
        log.append("3. Alice: Mã hóa tin nhắn thoại bằng AES-256 CBC.")

        # 4. Alice: Tạo hash (dấu vân tay số) của dữ liệu đã mã hóa (IV + ciphertext) bằng SHA-256.
        #    Dùng để kiểm tra tính toàn vẹn, đảm bảo dữ liệu không bị thay đổi trên đường truyền.
        data_to_hash = iv + ciphertext
        hasher = hashes.Hash(hashes.SHA256())
        hasher.update(data_to_hash)
        final_hash = hasher.finalize()
        log.append(f"4. Alice: Tạo hash SHA-256.")

        # 5. Alice: Dùng khóa riêng của mình để ký lên hash (tạo chữ ký số bằng RSA-PSS).
        #    Điều này chứng minh người gửi là Alice (xác thực) và chống chối bỏ.
        signature = alice_private_key.sign(
            final_hash,
            rsa_padding.PSS(mgf=rsa_padding.MGF1(hashes.SHA256()), salt_length=rsa_padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        log.append("5. Alice: Dùng khóa riêng của mình để ký lên hash.")

        # 6. Alice: Đóng gói tất cả (bản mã, iv, khóa AES đã mã hóa, hash, chữ ký) vào một gói tin JSON.
        #    Dữ liệu nhị phân được mã hóa sang Base64 để dễ dàng truyền tải trong JSON.
        packet = {
            "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'),
            "iv": base64.b64encode(iv).decode('utf-8'),
            "cipher": base64.b64encode(ciphertext).decode('utf-8'),
            "hash": final_hash.hex(),
            "sig": base64.b64encode(signature).decode('utf-8')
        }
        log.append("6. Alice: Đóng gói và gửi đi.")

        # =======================================================
        # Bắt đầu mô phỏng phía Người nhận (Bob)
        # =======================================================
        log.append("\n--- Bắt đầu phía Người nhận (Bob) ---")

        # Chuyển đổi chuỗi khóa thành đối tượng khóa.
        _, alice_public_key = deserialize_keys(None, alice_pem_public)
        bob_private_key_obj, _ = deserialize_keys(bob_pem_private, None)

        # 1. Bob: Dùng khóa riêng của mình để giải mã khóa AES mà Alice đã gửi.
        decrypted_aes_key = bob_private_key_obj.decrypt(
            base64.b64decode(packet['encrypted_aes_key']),
            rsa_padding.OAEP(mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        log.append("1. Bob: Dùng khóa riêng giải mã thành công khóa AES.")

        # 2. Bob: Bắt đầu kiểm tra chữ ký và tính toàn vẹn...
        log.append("2. Bob: Bắt đầu kiểm tra chữ ký và tính toàn vẹn...")
        # Tính toán lại hash từ dữ liệu nhận được.
        recalculated_hash_obj = hashes.Hash(hashes.SHA256())
        recalculated_hash_obj.update(base64.b64decode(packet['iv']) + base64.b64decode(packet['cipher']))
        recalculated_hash = recalculated_hash_obj.finalize()

        # So sánh hash tính lại với hash nhận được. Nếu khác, dữ liệu đã bị thay đổi.
        if recalculated_hash.hex() != packet['hash']:
            raise ValueError("Lỗi Toàn vẹn (Integrity Error)! Hash không khớp.")
        log.append("   - Kiểm tra Hash Toàn vẹn: OK.")

        # Dùng khóa công khai của Alice để xác thực chữ ký. Nếu thành công, chứng tỏ đúng là Alice đã gửi.
        alice_public_key.verify(
            base64.b64decode(packet['sig']),
            recalculated_hash,
            rsa_padding.PSS(mgf=rsa_padding.MGF1(hashes.SHA256()), salt_length=rsa_padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        log.append("   - Xác thực Chữ ký của Alice: OK.")

        # 3. Bob: Mọi thứ hợp lệ. Tiến hành giải mã tin nhắn thoại...
        log.append("3. Bob: Mọi thứ hợp lệ. Tiến hành giải mã tin nhắn thoại...")
        # Dùng khóa AES đã giải mã ở bước 1 để giải mã tin nhắn.
        decryptor_cipher = Cipher(algorithms.AES(decrypted_aes_key), modes.CBC(base64.b64decode(packet['iv'])))
        unpadder = PKCS7(algorithms.AES.block_size).unpadder()
        decrypted_padded_msg = decryptor_cipher.decryptor().update(base64.b64decode(packet['cipher']))
        decrypted_msg_bytes = unpadder.update(decrypted_padded_msg) + unpadder.finalize()

        log.append(f"   - Giải mã thành công! Nhận được {len(decrypted_msg_bytes)} bytes dữ liệu âm thanh.")
        log.append(f"\n✅ TIN NHẮN THOẠI ĐÃ ĐƯỢC GIẢI MÃ AN TOÀN! (Gửi ACK)")

        # Trả về kết quả thành công cùng gói tin và nhật ký cho frontend.
        return jsonify({"success": True, "packet": packet, "log": "\n".join(log)})

    except Exception as e:
        # Nếu có bất kỳ lỗi nào xảy ra trong quá trình xử lý.
        log.append(f"\n❌ NACK: ĐÃ XẢY RA LỖI: {str(e)}")
        # Trả về thông báo lỗi cho frontend.
        return jsonify({"success": False, "log": "\n".join(log)})

# ==============================================================================
# PHẦN 3: KHỞI CHẠY ỨNG DỤNG
# ==============================================================================
print("🚀 Bước 3: Khởi chạy ứng dụng web...")
# Khởi chạy một đường hầm ngrok tới cổng 5000 của máy cục bộ.
# Bất kỳ ai trên thế giới có URL này đều có thể truy cập vào ứng dụng Flask đang chạy.
public_url = ngrok.connect(5000)
print(f"🌍 Ứng dụng của bạn đang chạy tại URL công khai: {public_url}")

# Khởi chạy máy chủ web Flask, lắng nghe các kết nối trên cổng 5000.
app.run(port=5000)

⏳ Bước 1: Cài đặt các thư viện cần thiết...
✅ Cài đặt hoàn tất.
🔑 Bước 2: Cấu hình token ngrok...
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Token đã được cấu hình.
🚀 Bước 3: Khởi chạy ứng dụng web...
🌍 Ứng dụng của bạn đang chạy tại URL công khai: NgrokTunnel: "https://d038-34-147-102-236.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 06:44:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 06:44:42] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 06:44:43] "POST /generate-all-keys HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 06:44:56] "POST /process-message HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jun/2025 06:45:02] "GET /favicon.ico HTTP/1.1" 404 -
